In [1]:
!pip install Flask pymongo python-dateutil
!pip install --upgrade parso jedi

  Obtaining dependency information for jedi from https://files.pythonhosted.org/packages/20/9f/bc63f0f0737ad7a60800bfd472a4836661adae21f9c2535f3957b1e54ceb/jedi-0.19.1-py2.py3-none-any.whl.metadata
  Using cached jedi-0.19.1-py2.py3-none-any.whl.metadata (22 kB)
Using cached jedi-0.19.1-py2.py3-none-any.whl (1.6 MB)
  Attempting uninstall: jedi
    Found existing installation: jedi 0.18.0
    Uninstalling jedi-0.18.0:
      Successfully uninstalled jedi-0.18.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
spyder 5.4.3 requires pyqt5<5.16, which is not installed.
spyder 5.4.3 requires pyqtwebengine<5.16, which is not installed.
python-lsp-server 1.7.2 requires jedi<0.19.0,>=0.17.2, but you have jedi 0.19.1 which is incompatible.
ipywidgets 7.8.1 requires jupyterlab-widgets<3,>=1.0.0; python_version >= "3.6", but you have jupyterlab-widgets 3.0.10 which is inco

In [2]:
from flask import Flask, request, jsonify
from pymongo import MongoClient
from dateutil import parser
import re

In [3]:
app = Flask(__name__)

In [4]:
# MongoDB Setup
client = MongoClient('mongodb://localhost:27017/')  # Update with your MongoDB connection URI
db = client['news_database']
collection = db['news_articles']

In [5]:
# Function to clean and parse published_time
def normalize_published_time(published_time):
    # Remove unwanted text like "Updated"
    cleaned_time = re.sub(r"Updated|\s+", " ", published_time).strip()
    
    try:
        # Parse date using dateutil.parser
        parsed_date = parser.parse(cleaned_time)
        return parsed_date.isoformat()  # Convert to ISO format for MongoDB
    except (parser.ParserError, ValueError):
        return None

In [6]:
@app.route('/api/news', methods=['GET'])
def get_news():
    # Fetch query parameters for filters and pagination
    category = request.args.get('category')
    source = request.args.get('source')
    page = int(request.args.get('page', 1))
    limit = int(request.args.get('limit', 10))

    # Build MongoDB query
    query = {}
    if category:
        query["category"] = category
    if source:
        query["source"] = source

    # Calculate skip for pagination
    skip = (page - 1) * limit

    # Query the database with filters and pagination
    news_cursor = news_collection.find(query).skip(skip).limit(limit)
    news_list = []

    for news in news_cursor:
        # Parse and normalize the published_time field
        news['published_time'] = normalize_published_time(news.get('published_time', ''))
        news_list.append(news)

    return jsonify(news_list)

In [7]:
if __name__ == '__main__':
    app.run(debug=True)

 * Serving Flask app '__main__'
 * Debug mode: on


 * Running on http://127.0.0.1:5000
Press CTRL+C to quit
 * Restarting with watchdog (fsevents)
Traceback (most recent call last):
  File "/Users/dibyendu/anaconda3/lib/python3.11/site-packages/ipykernel_launcher.py", line 15, in <module>
    from ipykernel import kernelapp as app
  File "/Users/dibyendu/anaconda3/lib/python3.11/site-packages/ipykernel/kernelapp.py", line 19, in <module>
    from IPython.core.application import (  # type:ignore[attr-defined]
  File "/Users/dibyendu/anaconda3/lib/python3.11/site-packages/IPython/__init__.py", line 54, in <module>
    from .terminal.embed import embed
  File "/Users/dibyendu/anaconda3/lib/python3.11/site-packages/IPython/terminal/embed.py", line 16, in <module>
    from IPython.terminal.interactiveshell import TerminalInteractiveShell
  File "/Users/dibyendu/anaconda3/lib/python3.11/site-packages/IPython/terminal/interactiveshell.py", line 48, in <module>
    from .debugger import TerminalPdb, Pdb
  File "/Users/dibyendu/anaconda3/lib/py

SystemExit: 1

/Users/dibyendu/anaconda3/lib/python3.11/site-packages/IPython/core/interactiveshell.py:3534: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)
